In [ ]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt

In [ ]:
cat = fits.open('COLA1_O3_fitted_flux.fits')[1].data
cat.columns

In [ ]:
src = cat['NUMBER_1'] == 9269

In [ ]:
import astropy.units as u
import astropy.constants as c

# F356W without lines
fwhm = 0.787 * u.micron
w_pivot = 3.563 * u.micron
redshift = 6.5916

F356W = cat['F356W_AUTO_fnu'][src] * u.nJy

Fl_units = 1e-18 * u.erg * u.s**-1 * u.cm**-2

flambda_OIII_5008 = cat['f_O3_5008'][src] * Fl_units / w_pivot / fwhm * 0.5008 * u.micron * (1 + redshift)
flambda_OIII_4960 = cat['f_O3_4960'][src] * Fl_units / w_pivot / fwhm * 0.4960 * u.micron * (1 + redshift)
flambda_OIII_4363 = cat['f_O3_4363'][src] * Fl_units / w_pivot / fwhm * 0.4363 * u.micron * (1 + redshift)
flambda_Hb = cat['f_Hb'][src] * Fl_units / w_pivot / fwhm * 0.4862 * u.micron * (1 + redshift)
flambda_Hg = cat['f_Hg'][src] * Fl_units / w_pivot / fwhm * 0.4341 * u.micron * (1 + redshift)

flambda_lines = flambda_OIII_4363 + flambda_OIII_4960 + flambda_OIII_5008 + flambda_Hb + flambda_Hg


F356W_noline = F356W - flambda_lines / c.c * w_pivot**2
F356W_noline

In [ ]:
def double_power_law(wav, norm, wav_break, beta1, beta2):
    exp1 = (wav_break - wav) * (beta1 - 1)
    exp2 = (wav_break - wav) * (beta2 - 1)

    return norm * (10. ** exp1 + 10. ** exp2)

In [ ]:
fig, ax = plt.subplots()

# F115W
ax.errorbar(1.154, cat['F115W_AUTO_fnu'][src],
            yerr=cat['F115W_AUTO_enu'][src],
            fmt='o', c='C0')
# F150W
ax.errorbar(1.501, cat['F150W_AUTO_fnu'][src],
            yerr=cat['F150W_AUTO_enu'][src],
            fmt='o', c='C0')
# F200W
ax.errorbar(1.990, cat['F200W_AUTO_fnu'][src],
            yerr=cat['F200W_AUTO_enu'][src],
            fmt='o', c='C0')
# F356W
ax.errorbar(3.563, cat['F356W_AUTO_fnu'][src],
            yerr=cat['F356W_AUTO_enu'][src],
            fmt='o', c='C0')
ax.errorbar(3.563, F356W_noline.value,
            yerr=cat['F356W_AUTO_enu'][src],
            fmt='o', c='C1')

# Load MCMC
chain = np.load(f'mcmc_dpl_fit_chain.npy')
xx = np.linspace(1, 4, 1000)
chain_len = 10000
yy_list = []
for i, params in enumerate(chain[-chain_len:]):
    yy_list.append(double_power_law(xx, *params))
ax.fill_between(xx, np.percentile(yy_list, 16, axis=0),
                 np.percentile(yy_list, 84, axis=0), alpha=0.5, color='C0')
ax.plot(xx, np.median(yy_list, axis=0), c='C0')
ax.plot(xx, double_power_law(xx, 400, 1.5, -2, 2), c='C1')

ax.set_ylim(30, 600)

ax.plot(xx, 300 * xx**-1.2)

plt.show()

In [ ]:
wav1500 = 1500 * u.AA
redshift = 6.5916

beta_list = []

for iii in range(len(yy_list)):
    rest_wav = xx * u.micron / (1 + redshift)
    rest_flux = yy_list[iii] * (1 + redshift)

    # Covnert flux to flambda
    rest_flux = rest_flux * c.c / rest_wav**2

    flux1500 = np.interp(wav1500, rest_wav, rest_flux)

    dfdl = np.interp(wav1500, rest_wav, np.gradient(rest_flux, rest_wav))
    beta = dfdl * wav1500 / flux1500
    beta_list.append(beta.to(u.dimensionless_unscaled).value)

beta_percs = np.percentile(beta_list, [16, 50, 84])

print(beta_percs[1], -beta_percs[1] + beta_percs)

In [ ]:
# Chisholm+22
N_iter = 100000
a = np.random.normal(1.3, 0.6, N_iter)
b = np.random.normal(-1.22, 0.1, N_iter)
beta = np.random.choice(beta_list, replace=True, size=N_iter)

fesc_Arr = a * 1e-4 * 10.**(b * beta)

fesc_percs = np.percentile(fesc_Arr, [16, 50, 84])
print(fesc_percs[1], -fesc_percs[1] + fesc_percs)
print(np.percentile(fesc_Arr, [16]))

In [ ]:
from jpasLAEs.utils import flux_to_mag
from astropy.cosmology import Planck18 as cosmo

# Compute MUV
this_distmod = cosmo.distmod(redshift).value
flx_units = u.erg * u.cm**-2 * u.s**-1 * u.AA**-1

wav1500 = 1500 * (1 + redshift)
obs_wav = (xx * u.micron).to(u.AA).value

m_list = []

for iii in range(len(yy_list)):
    obs_flux = yy_list[iii]

    flux1500 = (np.interp(wav1500, obs_wav, obs_flux) * u.nJy * c.c / (1500 * (1 + redshift) * u.AA)**2).to(flx_units).value

    this_m_UV = flux_to_mag(flux1500, wav1500)
    this_m_UV[~np.isfinite(this_m_UV)] = -99

    m_list.append(this_m_UV[0])

this_M_UV_Arr = np.asarray(m_list) - this_distmod + 2.5 * np.log10(1 + redshift)

MUV_percs = np.percentile(this_M_UV_Arr, [16, 50, 86])
print(MUV_percs[1], -MUV_percs[1] + MUV_percs)